# Weekly Project 5 
## Implementation of global registration 
### Task 1
Today, your task is to implement a global registration algorithm.

It should be able to roughly align two point clouds.
Implement the global registration, and then try the following:

1. Can you fit `r1.pcd` and `r2.pcd`?
2. Can you fit `car1.ply` and `car2.ply`?
The corresponding files are in the `global_registration` folder.


### Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in `car_challange` folder.

You can use the notebooks from Monday as a starting point.

In [1]:
import open3d as o3d
import numpy as np
import copy
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# helper function for drawing
# If you want it to be more clear set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [3]:
# GLOBAL REGISTRATION
# read pointcloud
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")

# Used for downsampling.
voxel_size = 0.05
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)
source.estimate_normals()
target.estimate_normals()

# Show models side by side
draw_registrations(source, target)

In [3]:
# global registration with features
def global_registration(source, target, vs, tr, rf):
    point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
    radius_feature = vs * rf
    f_source = o3d.pipelines.registration.compute_fpfh_feature(source, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=1000))
    f_target = o3d.pipelines.registration.compute_fpfh_feature(target, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=1000))
    distance_threshold = vs * tr

    corr_length = 0.9


    c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
    c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

    checker_list = [c0, c1, c2]

    ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, 
        f_source, f_target, True,
        distance_threshold,
        point_to_point,
        checkers = checker_list,
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.999))

    return ransac_result

In [5]:
ransac_result = global_registration(source, target, voxel_size, 1.5, 5)
draw_registrations(source, target, ransac_result.transformation, True)


In [6]:
# NOW WITH THE CAR DATA
source = o3d.io.read_point_cloud("global_registration/car1.ply")
target = o3d.io.read_point_cloud("global_registration/car2.ply")

# Used for downsampling.
voxel_size = 0.05
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)

source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)

draw_registrations(source, target, None, True)
ransac_result = global_registration(source, target, voxel_size,0.9, 2)
draw_registrations(source, target, ransac_result.transformation, True)

CHALLENGE TIME

In [4]:
# Implement local registration

def local_registration(source, target, threshold):
    trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    # evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
    # print(evaluation)   
    point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        point_to_plane)
    
    return icp_result

    

In [22]:
# read pointcloud
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")
# Used for downsampling.
voxel_size = 0.05
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)
# source.estimate_normals()
# target.estimate_normals()
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)

# Show models side by side
draw_registrations(source, target)

# START WITH GLOBAL REGISTRATION 
ransac_result = global_registration(source, target, voxel_size,0.9, 2)
draw_registrations(source, target, ransac_result.transformation, True)
source = source.transform(ransac_result.transformation)
# DO LOCAL REGISTRATION
threshold = 0.3
res = local_registration(source, target, threshold)
draw_registrations(source, target, res.transformation, True)

RegistrationResult with fitness=7.420168e-01, inlier_rmse=6.027961e-02, and correspondence_set size of 3532
Access transformation to get result.


In [ ]:
# Attempt to reconstruct the car from the images in car_challange folder.
# idea: read the images, match in batches (global plus local alignment), then downsample 

In [5]:
def getPointCloud(path_color, path_depth, camera, T, voxel_size):
    color = o3d.io.read_image(path_color)
    depth = o3d.io.read_image(path_depth)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, 
        depth, 
        convert_rgb_to_intensity = False)
    pc = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, camera)
    pc.transform(T)
    pc = pc.voxel_down_sample(voxel_size=voxel_size)
    pc.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
    return pc

In [6]:
# recreate object pointcloud
camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
T = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]

## PARAMETERS
voxel_size = 0.5
batch_size = 10
jumps = 1
threshold = 0.2
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

## READ FILES
color_dir = 'car_challange/rgb/'
depth_dir = 'car_challange/depth/'
color_files = sorted(os.listdir(color_dir))
depth_files = sorted(os.listdir(depth_dir))

path_color = os.path.join(color_dir, color_files[0])
path_depth = os.path.join(depth_dir, depth_files[0])
match_pc = getPointCloud(path_color, path_depth, camera, T, voxel_size)
pc_prev = match_pc

images = []     # RGB images
pcs = []        # pointclouds 
indx = 1

while indx < 250:
    # match PC with prev PC and then add matches onto each other
# for indx in range(1, len(color_files)):    
    path_color = os.path.join(color_dir, color_files[indx])
    path_depth = os.path.join(depth_dir, depth_files[indx])
    pc = getPointCloud(path_color, path_depth, camera, T, voxel_size)

    ########### HERE THE open3d MAGIC HAPPENS ####################

    # do global registration 
    ransac_result = global_registration(pc, pc_prev, voxel_size, 0.9, 2)
    # transform 
    pc.transform(ransac_result.transformation)
    # do local registration 
    ransac_result = local_registration(pc, pc_prev, threshold)
    # downsample every x batches 
    pc.transform(ransac_result.transformation)
    # match_pc = match_pc.voxel_down_sample(voxel_size=voxel_size)
    # add matches but match with prev not total 
    pc_prev = pc
    pc_prev.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                    max_nn=30),fast_normal_computation=True)
    match_pc += pc
    if(indx % ((jumps+1) * batch_size) == 0):
        match_pc = match_pc.voxel_down_sample(voxel_size=voxel_size)
        # o3d.visualization.draw_geometries([match_pc])
        print("--Downsampling--")
    print("Match " + str(indx) + " of " + str(1722))
    indx += jumps

    ##############################################################


    # o3d.visualization.draw_geometries([pc])
    # o3d.visualization.draw_geometries([match_pc])

Match 1 of 1722
Match 2 of 1722
Match 3 of 1722
Match 4 of 1722
Match 5 of 1722
Match 6 of 1722
Match 7 of 1722
Match 8 of 1722
Match 9 of 1722
Match 10 of 1722
Match 11 of 1722
Match 12 of 1722
Match 13 of 1722
Match 14 of 1722
Match 15 of 1722
Match 16 of 1722
Match 17 of 1722
Match 18 of 1722
Match 19 of 1722
--Downsampling--
Match 20 of 1722
Match 21 of 1722
Match 22 of 1722
Match 23 of 1722
Match 24 of 1722
Match 25 of 1722
Match 26 of 1722
Match 27 of 1722
Match 28 of 1722
Match 29 of 1722
Match 30 of 1722
Match 31 of 1722
Match 32 of 1722
Match 33 of 1722
Match 34 of 1722
Match 35 of 1722
Match 36 of 1722
Match 37 of 1722
Match 38 of 1722
Match 39 of 1722
--Downsampling--
Match 40 of 1722
Match 41 of 1722
Match 42 of 1722
Match 43 of 1722
Match 44 of 1722
Match 45 of 1722
Match 46 of 1722
Match 47 of 1722
Match 48 of 1722
Match 49 of 1722
Match 50 of 1722
Match 51 of 1722
Match 52 of 1722
Match 53 of 1722
Match 54 of 1722
Match 55 of 1722
Match 56 of 1722
Match 57 of 1722
Match 

In [15]:
o3d.visualization.draw_geometries([match_pc])